In [642]:
import pandas as pd
import numpy as np

### Reading dataframes for each protein into python 
Renaming columns to be easily accessable, normalizing the excitation and emission spectra, taking sum of column 

In [643]:
df_mEmerald = pd.read_csv('mEmerald_fpbase_spectra.csv')
df_mEmerald.fillna(0, inplace = True)
df_mTagBFP2 = pd.read_csv('mTagBFP2_fpbase_spectra.csv')
df_mTagBFP2.fillna(0, inplace = True)
df_mCherry = pd.read_csv('mCherry_fpbase_spectra.csv')
df_mCherry.fillna(0, inplace = True)
df_mNeptune2p5 = pd.read_csv('mNeptune2p5_fbpase_spectra.csv')
df_mNeptune2p5.fillna(0, inplace = True)

mEmerald_columns = df_mEmerald.rename(columns ={'mEmerald ex': 'excitation', 'mEmerald em': 'emission'})
mTagBFP2_columns = df_mTagBFP2.rename(columns ={'mTagBFP2 ex': 'excitation', 'mTagBFP2 em': 'emission'})
mCherry_columns = df_mCherry.rename(columns ={'mCherry ex': 'excitation', 'mCherry em': 'emission'})
mNeptune2p5_columns = df_mNeptune2p5.rename(columns ={'mNeptune2.5 ex': 'excitation', 'mNeptune2.5 em': 'emission'})

spectra_list = [mEmerald_columns, mTagBFP2_columns, mCherry_columns, mNeptune2p5_columns]
for j in range(0,4):
    excitation_column = spectra_list[j]['excitation']
    area = np.trapz(excitation_column, dx = 1)
    normalized = excitation_column/area
    excitation_column = normalized
    spectra_list[j]['excitation'] = normalized
    new_area = np.trapz(excitation_column, dx = 1)
    column = spectra_list[j]['emission']
    area_emission = np.trapz(column, dx = 1)
    normalized_em = column/area_emission
    column = normalized_em
    spectra_list[j]['emission'] = normalized_em
    new_area_em = np.trapz(column, dx = 1)

### Making sure each spectra dataframe has data for 400-900nm

In [644]:
wavelength = 701
for index in range(401, 601):
    mEmerald_columns.loc[len(mEmerald_columns.index)] = [wavelength, 0.0, 0.0]
    wavelength = wavelength + 1

wavelength = 651
for index in range(341, 591):
    mTagBFP2_columns.loc[len(mTagBFP2_columns.index)] = [wavelength, 0.0, 0.0]
    wavelength = wavelength + 1

wavelength = 851
for index in range(621, 671):
    mNeptune2p5_columns.loc[len(mNeptune2p5_columns.index)] = [wavelength, 0.0, 0.0]
    wavelength = wavelength + 1

### Function to solve emission bin area using i and j 
Corresponding lambda_min and lambda_max to the min and max wavelengths in channel i in a list of the 200 total bins 

In [645]:
spectra_list = [mEmerald_columns, mTagBFP2_columns, mCherry_columns, mNeptune2p5_columns]

def calc_emission_bin(i, j):
    bins_list = [*range(400, 900, 10)]
    total_bins = 4*bins_list 
    df_spectra = spectra_list[j]
    lambda_min = total_bins[i]
    lambda_max = lambda_min + 10
    bin_i = range(lambda_min, lambda_max)
    df_sliced_data = df_spectra.loc[df_spectra['wavelength'].isin(bin_i)]
    # print(df_sliced_data)
    sliced_data_emmision = df_sliced_data['emission']
    emission_bin = np.trapz(sliced_data_emmision)
    print('Emission bin sum for i = ', i, 'and j = ', j, 'is', emission_bin)
    return emission_bin

### Function to calculate the excitation values from the illumination wavelengths
Figure out how to go into csv and grab the specific excitation values from the dataframe that you found earlier. Can use for loop and/or a function. 

Excitation value corresponding to excitation wavelengths for each protein: 405nm, 488nm, 561nm, 637nm. Found these values on csv files

In [646]:
mEmerald_excitation = [0.1732, 0.9982, 0.0043, 0.0027]
mTag_excitation = [0.9588, 0, 0, 0]
mCherry_excitation = [0.0198, 0.0762, 0.6394, 0.0067]
mNeptune_excitation = [0.0279, 0.0717, 0.5819, 0.1519]

In [647]:
def calc_excitation(i, j): 
    illumation_wavelength_list = [405, 488, 561, 637]
    if 0 <= i < 50: 
        illumation_wavelength = illumation_wavelength_list[0]
    elif 50 <= i < 100: 
        illumation_wavelength = illumation_wavelength_list[1]
    elif 100 <= i < 150:
        illumation_wavelength = illumation_wavelength_list[2]
    elif 150 <= i < 200:
        illumation_wavelength = illumation_wavelength_list[3]
    else:
        print('Invalid i value')
    df_sliced = spectra_list[j].loc[spectra_list[j]['wavelength'] == illumation_wavelength]
    excitation_wavelength = df_sliced['excitation'].iloc[0]
    return excitation_wavelength


calc_excitation(2, 3)

0.0003359525187106889

### Function to index over brightness values using a given j 
Found brightness values on slack 

In [648]:
def calc_brightness(j):
    brightness_list = [39.1, 32.38, 15.85, 22.8]
    brightness = brightness_list[j]
    print('Brightness value for j = ', j, 'is', brightness)
    return brightness

### Function for the variable $k_i$
We are trying to give each fluorophore a fair chance. Each of our illumination wavelengths is targetted mainly at a specific fluorophore. Experimentally, we can choose the illumination strengths. So, we want to balance out our illumination strengths so that a similar amount of information (photons) is collected from each fluorophore emissions.

Depends on the channel--each set of 50 channels will have a different $k_i$. Could make it depend on the brightnesses.

In [649]:
def calc_k(i, j):
    spectra_list = [mEmerald_columns, mTagBFP2_columns, mCherry_columns, mNeptune2p5_columns]
    illumination_wavelengths = [405, 488, 561, 637]
    brightness_list = [39.1, 32.38, 15.85, 22.8]
    brightness = brightness_list[j]
    method = 'small if large b'
    if method == 'small if large b':
        m = 5
        if 0 <= i < 50:
            k = 1 / (.95 * m)
        elif 50 <= i < 100:
            k = 1 / (.94 * m)
        elif 100 <= i < 150:
            k = 1 / (.92 * m)
        elif 150 <= i < 200:
            k = 1 / (.93 * m)
    elif method == 'scale by b':
        k = 1 / brightness
    elif method == 'using max aij/make "K" all ones':
        if 0 <= i < 50:
            k = 1 / calc_a_constant_k(5 ,1)
        elif 50 <= i < 100:
            k = 1 / calc_a_constant_k(61 ,0)
        elif 100 <= i < 150:
            k = 1 / calc_a_constant_k(124 ,3)
        elif 150 <= i < 200:
            k = 1 / calc_a_constant_k(174 ,3)
    elif method == 'using good k value':
        if 0 <= i < 50:
            k = 1 / calc_a_constant_k(i ,j)
        elif 50 <= i < 100:
            k = 1 / (.94 * m)
        elif 100 <= i < 150:
            k = 1 / (.92 * m)
        elif 150 <= i < 200:
            k = 1 / (.93 * m)
        # not completed yet, needs replacing of code
    else:
        k = 1
    return k

calc_k(10, 0)

B = np.zeros((200, 4)) ##200 rows (horizontal), 4 columns (vertical)

for j in range(0, 4):
    for i in range(0, 200):
        B[i, j] = calc_k(i, j)

print(B)

[[0.21052632 0.21052632 0.21052632 0.21052632]
 [0.21052632 0.21052632 0.21052632 0.21052632]
 [0.21052632 0.21052632 0.21052632 0.21052632]
 [0.21052632 0.21052632 0.21052632 0.21052632]
 [0.21052632 0.21052632 0.21052632 0.21052632]
 [0.21052632 0.21052632 0.21052632 0.21052632]
 [0.21052632 0.21052632 0.21052632 0.21052632]
 [0.21052632 0.21052632 0.21052632 0.21052632]
 [0.21052632 0.21052632 0.21052632 0.21052632]
 [0.21052632 0.21052632 0.21052632 0.21052632]
 [0.21052632 0.21052632 0.21052632 0.21052632]
 [0.21052632 0.21052632 0.21052632 0.21052632]
 [0.21052632 0.21052632 0.21052632 0.21052632]
 [0.21052632 0.21052632 0.21052632 0.21052632]
 [0.21052632 0.21052632 0.21052632 0.21052632]
 [0.21052632 0.21052632 0.21052632 0.21052632]
 [0.21052632 0.21052632 0.21052632 0.21052632]
 [0.21052632 0.21052632 0.21052632 0.21052632]
 [0.21052632 0.21052632 0.21052632 0.21052632]
 [0.21052632 0.21052632 0.21052632 0.21052632]
 [0.21052632 0.21052632 0.21052632 0.21052632]
 [0.21052632 

### Function to solve for matrix A just inputting i and j value 
All above functions called in this function to reduce reduncancy in solving for the matrix value

In [650]:
def calc_a(i, j):
    k = calc_k(i, j)
    brightness_value = calc_brightness(j)
    excitation_value = calc_excitation(i, j)
    emission_bin_area = calc_emission_bin(i, j)
    a = k * brightness_value * excitation_value * emission_bin_area
    print('Matrix A for image channel i = ', i, ' and spectrum j = ', j, 'is ', a)
    return a

calc_a(15, 2)

Brightness value for j =  2 is 15.85
Emission bin sum for i =  15 and j =  2 is 0.000348535231102944
Matrix A for image channel i =  15  and spectrum j =  2 is  3.006382787419129e-07


3.006382787419129e-07

### For loop for matrix A 

In [651]:
A = np.zeros((200, 4)) ##199 rows (horizontal), 4 columns (vertical)

for j in range(0, 4):
    for i in range(0, 200):
        A[i, j] = calc_a(i, j)

print(A)

Brightness value for j =  0 is 39.1
Emission bin sum for i =  0 and j =  0 is 0.0
Matrix A for image channel i =  0  and spectrum j =  0 is  0.0
Brightness value for j =  0 is 39.1
Emission bin sum for i =  1 and j =  0 is 0.0
Matrix A for image channel i =  1  and spectrum j =  0 is  0.0
Brightness value for j =  0 is 39.1
Emission bin sum for i =  2 and j =  0 is 0.0
Matrix A for image channel i =  2  and spectrum j =  0 is  0.0
Brightness value for j =  0 is 39.1
Emission bin sum for i =  3 and j =  0 is 0.0
Matrix A for image channel i =  3  and spectrum j =  0 is  0.0
Brightness value for j =  0 is 39.1
Emission bin sum for i =  4 and j =  0 is 0.0
Matrix A for image channel i =  4  and spectrum j =  0 is  0.0
Brightness value for j =  0 is 39.1
Emission bin sum for i =  5 and j =  0 is 0.0
Matrix A for image channel i =  5  and spectrum j =  0 is  0.0
Brightness value for j =  0 is 39.1
Emission bin sum for i =  6 and j =  0 is 0.0005829611700132957
Matrix A for image channel i =

### How to save array as csv

In [652]:
np.savetxt("Matrix A.csv", A, delimiter=",")

Turning dataframe into csv

In [653]:
# mEmerald_columns.to_csv('mEmerald_columns 1')

### Create a vector consisting of some concentration of each fluorophore
The concentrations don't have to all be the same. Modeling our imaging system -- using one of each fluorophore in this example. Each index corresponds to a fluorophore

In [654]:
x = [1, 1, 1, 1] ##might have to switch orientation if error (transpose)

### Calculating the measurements of y using our imaging model
Multiplying Matrix A by Vector x

In [655]:
y = A @ x
y_reshaped = np.reshape(multiplied, (199, 1))
print(y_reshaped)

[[1.54887170e-05]
 [2.45799204e-04]
 [1.29544037e-03]
 [5.26717391e-03]
 [1.35022329e-02]
 [1.96649465e-02]
 [1.83116210e-02]
 [1.54206232e-02]
 [1.29598813e-02]
 [1.09355010e-02]
 [1.01647454e-02]
 [8.41439190e-03]
 [5.66150583e-03]
 [3.95653024e-03]
 [2.88369983e-03]
 [1.98670848e-03]
 [1.30038640e-03]
 [8.64510678e-04]
 [6.17840548e-04]
 [4.89342950e-04]
 [4.16296158e-04]
 [3.67761893e-04]
 [3.31894355e-04]
 [3.10759327e-04]
 [2.91125935e-04]
 [2.43474266e-04]
 [2.18145429e-04]
 [1.90297267e-04]
 [1.63564840e-04]
 [1.33846949e-04]
 [7.79233173e-05]
 [5.80025177e-05]
 [4.57744065e-05]
 [3.67327171e-05]
 [2.97975829e-05]
 [2.37676937e-05]
 [1.85031733e-05]
 [1.43663240e-05]
 [1.10884062e-05]
 [8.52991369e-06]
 [5.21186192e-06]
 [4.05618076e-06]
 [3.38584920e-06]
 [2.93866157e-06]
 [2.49507303e-06]
 [1.59259981e-07]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.000000

### Solving the Fisher Information Matrix 
Transposing A using simple command '.T'

In [656]:
transposed_A = A.T
transposed_A

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 1.23753011e-05, 8.06203823e-05,
        4.66884165e-04, 1.74564115e-03, 3.75724272e-03, 3.94634600e-03,
        2.55859952e-03, 1.85320736e-03, 1.48679713e-03, 1.04833033e-03,
        6.63031481e-04, 4.26067383e-04, 2.94739626e-04, 2.09415182e-04,
        1.44137278e-04, 9.90265318e-05, 6.93547574e-05, 5.25166285e-05,
        4.25054200e-05, 3.46894404e-05, 3.13362886e-05, 2.84897282e-05,
        2.87309621e-05, 2.82002474e-05, 1.54389721e-06, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.000000

Creating diagonal matrix of 1/y

In [657]:
FIM_y = 1/y
FIM_y[FIM_y == np.inf] = 0
diagonal_matrix = np.diag(FIM_y)
print(diagonal_matrix)

[[64563.12690457     0.             0.         ...     0.
      0.             0.        ]
 [    0.          4068.36142138     0.         ...     0.
      0.             0.        ]
 [    0.             0.           771.93827004 ...     0.
      0.             0.        ]
 ...
 [    0.             0.             0.         ...     0.
      0.             0.        ]
 [    0.             0.             0.         ...     0.
      0.             0.        ]
 [    0.             0.             0.         ...     0.
      0.             0.        ]]


/var/folders/jt/_z0pnbx50rs5xql_tz09rgq00000gn/T/ipykernel_56975/2314514205.py:1: RuntimeWarning: divide by zero encountered in true_divide
  FIM_y = 1/y


Code to multiply all three matrices and compute the fisher information matrix

In [658]:
FIM = transposed_A @ diagonal_matrix @ A
print(FIM)

[[1.17499601e-01 1.09243375e-02 1.50315026e-03 1.28389067e-03]
 [1.09243375e-02 1.04467570e-01 1.36376880e-04 1.02956300e-04]
 [1.50315026e-03 1.36376880e-04 1.49206153e-02 1.33898988e-02]
 [1.28389067e-03 1.02956300e-04 1.33898988e-02 2.97596801e-02]]


Inversing the fisher information matrix. 
If you inverse the fisher information matrix and multiply it by the initial fisher information matrix, should get a diagonal matrix of all 1's on the diagonal (and the rest of the elements should be 0. Ours are very small numbers that would round to zero). 

In [659]:
FIM_inverse = np.linalg.inv(FIM)
print(FIM_inverse) ###The CRLB numbers are the diagonal elements in this matrix

idk = FIM_inverse @ FIM ##Checking to make sure you inversed it right
print(idk.shape)
print(idk)

FIM_inverse[0,0] ##Can use a for loop to get all four or you can write them out

[[ 8.60522784e+00 -8.98735483e-01 -8.86131855e-01  3.05639918e-02]
 [-8.98735483e-01  9.66632927e+00 -4.35247299e-03  7.28999012e-03]
 [-8.86131855e-01 -4.35247299e-03  1.12501779e+02 -5.05801569e+01]
 [ 3.05639918e-02  7.28999012e-03 -5.05801569e+01  5.63589120e+01]]
(4, 4)
[[ 1.00000000e+00 -1.86724178e-17 -3.41523684e-18 -3.46944695e-18]
 [ 1.38168014e-17  1.00000000e+00  1.62630326e-19  8.13151629e-20]
 [ 1.38777878e-17  1.73472348e-18  1.00000000e+00  2.22044605e-16]
 [-1.38777878e-17  0.00000000e+00 -1.11022302e-16  1.00000000e+00]]


8.605227839440685

In [660]:
CRLB_0 = FIM_inverse[0,0]
CRLB_1 = FIM_inverse[1,1]
CRLB_2 = FIM_inverse[2,2]
CRLB_3 = FIM_inverse[3,3]
CRLB_list = [CRLB_0, CRLB_1, CRLB_2, CRLB_3]
print(CRLB_list)

[8.605227839440685, 9.666329265146565, 112.50177884419057, 56.35891200950882]


In [661]:
calc_FOM = [1/i for i in CRLB_list]
print(calc_FOM)

[0.11620842802286535, 0.10345188670591365, 0.008888748340459138, 0.017743422723122848]
